In [1]:
import pandas as pd
from transformers import AutoTokenizer
import re
from random import randint
import json
import os

#length limitations
max_input_len = 300
data_size = None #1000

INPUT = '../data/acl_arc/'
OUTPUT = f'../data/acl_arc/'
if data_size: OUTPUT = f'../data/acl_arc/{data_size}/'

# as ther is not such a hughe difference between the token count we use mistrals tokenizer for the evaluation
tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-Instruct-v0.3')


/home/explorer/anaconda3/envs/lasse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load data 
train_raw = pd.read_csv(INPUT + 'train_raw.txt',sep='\t')
test_raw = pd.read_csv(INPUT + 'test_raw.txt',sep='\t')

#extract relevant data
train_df = train_raw[['cite_context_paragraph','citation_class_label', 'section_title']] # citation_context
train_df.columns = ['text', 'label', 'section_title']
test_df = test_raw[['cite_context_paragraph','citation_class_label', 'section_title']] # citation_context
test_df.columns = ['text', 'label', 'section_title']


In [3]:
#helper 
def preprocess_df(df, max_len):
    res_df = pd.DataFrame(columns=['text', 'label','section_title'])
    for idx, row in df.iterrows():
        
        # load text and replace citations
        text = eval(row['text']) # for whole paragraph
        text = [replace_citations(sent) for sent in text]
        
        #find the location of the target citation
        target_location = [idx for idx, sent in enumerate(text) if '#TARGET_REF' in sent]
        if len(target_location) != 1: continue # ignore entries that have != 1 target ref
        
        # check the len of tokenized strin and restrict to < max_len
        sent_len = [len(tokenizer.encode(sent)) for sent in text] # get len of token
        total_len = sum(sent_len)
        
        # if total len > max len add sentences until got max len
        if total_len > max_len:
            # add citance
            cumm_len = sent_len[target_location[0]] 
            res_sent_ids = [target_location[0]]
            
            # add sentences
            while len(res_sent_ids) < len(text):
                chance = randint(0,1) # randomly front or bag sentence
                prev = res_sent_ids[0] - 1
                next = res_sent_ids[-1] +1
                if chance == 0 and prev >= 0:
                    if cumm_len + sent_len[prev] <= max_len:
                        res_sent_ids.insert(0, prev)
                        cumm_len += sent_len[prev]
                    else: break
                if chance == 1 and next < len(text):
                    if cumm_len + sent_len[next] <= max_len:
                        res_sent_ids.append(next)
                        cumm_len += sent_len[next]
                    else: break   
            text = [sent for idx, sent in enumerate(text) if idx in res_sent_ids]
        assert len(tokenizer.tokenize(' '.join(text))) <= max_len
        
        #prepare labels
        label = label_mapping(row['label'])
        
        
        # add text and labels to result
        res_df.loc[len(res_df)] = [text, label, row['section_title']]
    return res_df

def replace_citations(text):
    #replace and fix #AUTHOR_TAG
    text = re.sub(r'#AUTHOR_TAG', '#TARGET_REF', text)
    rext = re.sub(r'(?:van |)(?:[A-Z][a-z]+-?)+ ?(?:and [A-Za-z-]{2,}|et al?t?\.)?(?: |,)*?#TARGET_REF', '#TARGET_REF', text) #fix wrong parsing
    
    #replace all other citatons with #REF
    text = re.sub(r'(?:van |)(?:[A-Z][a-z]+-?)+ ?(?:and [A-Za-z-]{2,}|et al?t?\.)?(?: |,)*?(?:\d{4}|9\d|\[ ?\d{1,2} ?\]|\( ?\d{4} ?\))', '#REF', text)
    
    return text

def label_mapping(label):
    if type(label) == int:
        if label == 0: return 'BACKGROUND'
        if label == 1: return 'USE'
        if label == 2: return 'COMPARISON'
        if label == 3: return 'MOTIVATION'
        if label == 4: return 'EXTENSION'
        if label == 5: return 'FUTURE'
    else:
        if label == 'BACKGROUND': return 0
        if label == 'USE': return 1
        if label == 'COMPARISON': return 2
        if label == 'MOTIVATION': return 3
        if label == 'EXTENSION': return 4
        if label == 'FUTURE': return 5

In [4]:
#create different task shemata for acl-arc
'''data schema
    [{
        "gold": {
            "text": [<str>],
            "label": [<str>]
        },
        "input": <str>
        "output": <str>
    }]
'''

def create_xml_data(df):
    res_data = []
    for idx, row in df.iterrows():
        text = row['text']
        label = row['label']
        section_title = row['section_title']
        input = f'{section_title}:' + ' '.join(row['text'])
        
        #set XML tags
        tag_location = input.find('#TARGET_REF') + len('#TARGET_REF')
        output = input[:tag_location] + f'<{label}/>' + input[tag_location:]
        
        # add example to response
        res_data.append({
            "gold": {
                "text": text,
                "label": [label]
            },
            "input": input,
            "output": output
        })
    return res_data

def create_json_data(df):
    res_data = []
    for idx, row in df.iterrows():
        text = row['text']
        label = row['label']
        section_title = row['section_title']
        input = f'{section_title}:' + ' '.join(row['text'])
        
        #create json object
        output = {"label": [label]}
        
        # add example to response
        res_data.append({
            "gold": {
                "text": text,
                "label": [label]
            },
            "input": input,
            "output": json.dumps(output)
        })
    return res_data

# def create_json_2_data(df):
#     res_data = []
#     for idx, row in df.iterrows():
#         text = row['text']
#         label = row['label']
#         input = [f'sent{idx}: {sent}\n' for idx, sent in enumerate(text)]
        
#         #create json object
#         output = {"label": [label]}
        
#         # add example to response
#         res_data.append({
#             "gold": {
#                 "text": text,
#                 "label": [label]
#             },
#             "input": ' '.join(input),
#             "output": json.dumps(output)
#         })
#     return res_data

In [5]:
# prepare data
test_df_clean = preprocess_df(test_df, max_input_len)
train_df_clean = preprocess_df(train_df, max_input_len)

# create train data with size data_size
if data_size and data_size < len(train_df_clean):
    
    #create label specific df (reset index, shuffle)
    train_0, train_1, train_2, train_3, train_4, train_5 = train_df_clean[train_df_clean['label']== label_mapping(0)].sample(frac=1).reset_index(drop=True), train_df_clean[train_df_clean['label']== label_mapping(1)].sample(frac=1).reset_index(drop=True), train_df_clean[train_df_clean['label']== label_mapping(2)].sample(frac=1).reset_index(drop=True), train_df_clean[train_df_clean['label']== label_mapping(3)].sample(frac=1).reset_index(drop=True), train_df_clean[train_df_clean['label']== label_mapping(4)].sample(frac=1).reset_index(drop=True),train_df_clean[train_df_clean['label']== label_mapping(5)].sample(frac=1).reset_index(drop=True)
    
    #create new train_df
    train_df_clean = pd.DataFrame(columns=['text', 'label', 'section_title'])
    
    #add new samples until data_size is succeeded
    i = 0
    while len(train_df_clean) < data_size:
        if i < len(train_0): train_df_clean.loc[len(train_df_clean)] = train_0.loc[i]
        if i < len(train_1): train_df_clean.loc[len(train_df_clean)] = train_1.loc[i]
        if i < len(train_2): train_df_clean.loc[len(train_df_clean)] = train_2.loc[i]
        if i < len(train_3): train_df_clean.loc[len(train_df_clean)] = train_3.loc[i]
        if i < len(train_4): train_df_clean.loc[len(train_df_clean)] = train_4.loc[i]
        if i < len(train_5): train_df_clean.loc[len(train_df_clean)] = train_5.loc[i]
        i += 1
    train_df_clean = train_df_clean[:data_size]

# create XML data
test_xml = create_xml_data(test_df_clean)
train_xml = create_xml_data(train_df_clean)
XML_OUTPUT = OUTPUT + 'XML/'
os.makedirs(XML_OUTPUT, exist_ok=True)
with open(XML_OUTPUT+ 'train.json', 'w', encoding='utf-8') as f:
    json.dump(train_xml, f, ensure_ascii=False, indent=4)
with open(XML_OUTPUT+ 'test.json', 'w', encoding='utf-8') as f:
    json.dump(test_xml, f, ensure_ascii=False, indent=4)

# create JSON 1 data
test_json1 = create_json_data(test_df_clean)
train_json1 = create_json_data(train_df_clean)
JSON1_OUTPUT = OUTPUT + 'JSON/'
os.makedirs(JSON1_OUTPUT, exist_ok=True)
with open(JSON1_OUTPUT + '/train.json', 'w', encoding='utf-8') as f:
    json.dump(train_json1, f, ensure_ascii=False, indent=4)
with open(JSON1_OUTPUT + 'test.json', 'w', encoding='utf-8') as f:
    json.dump(test_json1, f, ensure_ascii=False, indent=4)

# create JSON 2 data
# test_json2 = create_json_2_data(test_df_clean)
# train_json2 = create_json_2_data(train_df_clean)
# JSON2_OUTPUT = OUTPUT + 'JSON2/'
# os.makedirs(JSON2_OUTPUT, exist_ok=True)
# with open(JSON2_OUTPUT + 'train.json', 'w', encoding='utf-8') as f:
#     json.dump(train_json2, f, ensure_ascii=False, indent=4)
# with open(JSON2_OUTPUT + 'test.json', 'w', encoding='utf-8') as f:
#     json.dump(test_json2, f, ensure_ascii=False, indent=4)
